# Polymorphic Variants

- more flexible and lightweight than ordinary variants
- explicit type declaration is not needed
- the type will be inferred automatically
- have leading backtick `` `Int 3 ``
- `>` at the beginning of the variant type means the type is open to combination with other variant types
- `>` means lower bound, `<` means upper bound. If the same set of tags are both an upper and a lower bound, we end up with an _exact polymorphic variant type_, which has neither marker.

In [5]:
let one = `Int 1 ;;
let two = `Float 2. ;;
let nan = `Not_a_number ;;

(* for the below line, compiler infers a new type *)
[one; two; nan] ;;

val one : [> `Int of int ] = `Int 1


val two : [> `Float of float ] = `Float 2.


val nan : [> `Not_a_number ] = `Not_a_number


- : [> `Float of float | `Int of int | `Not_a_number ] list =
[`Int 1; `Float 2.; `Not_a_number]


OCaml will in some cases infer a variant type with `<`, to indicate “these tags or less,” as in the following example:

In [3]:
let is_positive = function
     | `Int x -> x > 0
     | `Float x -> x > 0. ;;

val is_positive : [< `Float of float | `Int of int ] -> bool = <fun>


Compiler will show error for incompatible uses of the same tag

In [7]:
let one = `Int 1 ;;
let two = `Int "two" ;;
[one; two] ;;

val one : [> `Int of int ] = `Int 1


val two : [> `Int of string ] = `Int "two"


error: compile_error

Polymorphic variants can lead to farily complex inferred types: e.g.<br/>`` [< `Float of float | `Int of int | `Nan > `Float `Int ] ``

In [12]:
let is_positive = function
    | `Int x -> Ok (x > 0)
    | `Float x -> Ok (x > 0.) 
    | `Nan -> Error "Not a number" ;;
    
[(`Int 10); (`Float 20.)] |> List.filter  (fun x ->
    match is_positive x with
    | Ok b -> b
    | Error _ -> false 
) ;;

val is_positive :
  [< `Float of float | `Int of int | `Nan ] -> (bool, string) result = <fun>


- : [< `Float of float | `Int of int | `Nan > `Float `Int ] list =
[`Int 10; `Float 20.]


In [20]:
let f1 = function `A x -> x = 1 | `B -> true | `C -> false ;;
let f2 = function `A x -> x = "a" | `B -> true ;;
let f x = f1 x && f2 x ;;

f `B ;;

val f1 : [< `A of int | `B | `C ] -> bool = <fun>


val f2 : [< `A of string | `B ] -> bool = <fun>


val f : [< `A of string & int | `B ] -> bool = <fun>


- : bool = true


Type definitions as abbreviations.

In [24]:
type tags = [ `Tag1 of int | `Tag2 of float ] ;;
let f = function
    | #tags -> "tags"
    | `TagN -> "TagN" ;;

type tags = [ `Tag1 of int | `Tag2 of float ]


val f : [< `Tag1 of int | `Tag2 of float | `TagN ] -> string = <fun>


Subtyping

In [27]:
type ab = [`A | `B ] ;;
type abc = [ `A | `B | `C ] ;;
let x = `A ;;
let y : abc = (x :> abc) ;;

type ab = [ `A | `B ]


type abc = [ `A | `B | `C ]


val x : [> `A ] = `A


val y : abc = `A
